In [11]:
import sklearn
from sklearn.linear_model import Ridge
import time
from scipy.io import loadmat
import functionList
import numpy as np
import pandas as pd
import torch

In [12]:
raw = loadmat('sampleData.mat')
X = raw['data'][0][0]['data']
l = X.shape[0]

In [13]:
holdStack = X
for i in range(9):
    X = np.hstack((X,holdStack))

In [18]:
memVec = [20]
ridgeVec = [.1]
ridgeVecNormal = np.dot(ridgeVec,l)

s = (len(memVec),len(ridgeVec),10)
error = np.zeros(s)
runTimes = np.zeros(s)
for memNumber in memVec:
    featureMat = functionList.buildFeatureMat(X,memNumber)
    for regularizer in ridgeVecNormal: 
        RR = Ridge(alpha=regularizer)
        crossVal = sklearn.model_selection.KFold(n_splits=2)
        cvRun = 0
        for train, test in crossVal.split(featureMat):
            start = time.time()
            RR.fit(featureMat[train,:],X[train])
            end = time.time()
            pred = RR.predict(featureMat[test])
            error[memVec.index(memNumber), ridgeVec.index(regularizer/l), cvRun] = error[memVec.index(memNumber), ridgeVec.index(regularizer/l), cvRun] + functionList.findMSE(pred, X[test])
            runTimes[memVec.index(memNumber), ridgeVec.index(regularizer/l), cvRun] = end - start
            cvRun = cvRun + 1


In [19]:
print(runTimes)
print()
print(error)

[[[ 9.99572396 10.19354916  0.          0.          0.
    0.          0.          0.          0.          0.        ]]]

[[[0.44337395 0.44511422 0.         0.         0.         0.
   0.         0.         0.         0.        ]]]


In [20]:
errorGpu = np.zeros(s)
runTimesGpu = np.zeros(s)
for memNumber in memVec:
    featureMat = functionList.buildFeatureMat(X,memNumber)
    for regularizer in ridgeVecNormal: 
        crossVal = sklearn.model_selection.KFold(n_splits=2)
        cvRun = 0
        for train, test in crossVal.split(featureMat):
            start = time.time()
            functionList.solveRrClosedForm(featureMat[train,:],X[train],regularizer)
            end = time.time()
            #errorGpu[memVec.index(memNumber), ridgeVec.index(regularizer/l), cvRun] = errorGpu[memVec.index(memNumber), ridgeVec.index(regularizer/l), cvRun] + functionList.findMSE(pred, X[test])
            runTimesGpu[memVec.index(memNumber), ridgeVec.index(regularizer/l), cvRun] = end - start
            cvRun = cvRun + 1

In [21]:
print(runTimesGpu)
print()
print(errorGpu)

[[[13.25056863 14.77251196  0.          0.          0.
    0.          0.          0.          0.          0.        ]]]

[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]
